<a href="https://colab.research.google.com/github/Simon-creator-jara/Inteligencia_artificial/blob/main/Neurona_Final__def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final, Programación Orientada a Objetos

## Simón Jaramillo, Simón Gómez 

In [ ]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import math
import plotly.express as px
import matplotlib.pyplot as plt

# Construcción y limpieza de tablas

In [ ]:
#Descargamos dos bases de datos: La primera, data, clasifica la satisfacción de un cliente según ciertos puntajes,
data=pd.read_csv('test.csv')

FileNotFoundError: ignored

In [ ]:
#En este caso, estamos cambiando los nombres de las variables, y aquellas cualitativas, las estamos convirtiendo en dicotómicas.
data.rename(columns={'Gender':'Genero',
                      'Age':'Edad',
                      'Flight Distance': 'Distancia vuelo',
                      'Departure/Arrival time convenient':'Tiempo de espera',
                      'Ease of Online booking':'Booking online',
                      'Food and drink':'Alimentación',
                      'Online boarding':'Check-in',
                      'Seat comfort':'Comodidad',
                      'Inflight entertainment':'Entretenimiento',
                      'Baggage handling':'Manejo equipajes',
                      'Inflight service':'Servicio',
                      'Cleanliness':'Limpieza',
                      'satisfaction':'Satisfaccion'},inplace=True)
#Por motivos prácticos, hay qué cambiar las variables de genero y satisfacción, por 1 y 0, tal como se observa.
data.loc[data["Genero"] == "Male", "Genero"] = 1
data.loc[data["Genero"] == "Female", "Genero"] = 0
data.loc[data["Satisfaccion"] == "satisfied", "Satisfaccion"] = 1
data.loc[data["Satisfaccion"] == "neutral or dissatisfied", "Satisfaccion"] = 0

In [ ]:
#Ya con la base de datos para clasificación construida, tomamos las variables que nos interesan y las normalizamos, finalmente mostramos la tabla.
df=data.iloc[:,[2,4,7,9,10,12,13,14,15,18,20,21,24]]
df.insert(0,'bias',np.ones(df.shape[0]))
df=df/df.max()
df.head()

,bias,Genero,Edad,Distancia vuelo,Tiempo de espera,Booking online,Alimentación,Check-in,Comodidad,Entretenimiento,Manejo equipajes,Servicio,Limpieza,Satisfaccion
0,1.0,0.0,0.611765,0.032109,0.8,0.6,0.6,0.8,0.6,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,0.423529,0.574553,0.2,0.6,1.0,0.8,1.0,0.8,0.8,0.8,1.0,1.0
2,1.0,1.0,0.235294,0.038531,0.0,0.4,0.4,0.4,0.4,0.4,0.6,0.4,0.4,0.0
3,1.0,1.0,0.517647,0.677704,0.0,0.0,0.6,0.8,0.8,0.2,0.2,0.2,0.8,1.0
4,1.0,0.0,0.576471,0.237207,0.6,0.8,0.8,0.2,0.4,0.4,0.4,0.4,0.8,1.0


In [ ]:
#Convierto el DataFrame de Pandas a Numpy
df=df.to_numpy()

In [ ]:
data_r=pd.read_csv('Proyecto_medical.csv')

In [ ]:
#En este caso, estamos cambiando los nombres de las variables, y aquellas cualitativas, las estamos convirtiendo en dicotómicas.
data_r.rename(columns={'age':'Edad',
                      'sex': 'Genero',
                      'bmi':'BMI',
                      'children':'Número de hijos',
                      'smoker':'Fumador',
                      'region':'Region',
                      'charges':'Cobro'},inplace=True)
#Por motivos prácticos, hay qué cambiar las variables de genero y satisfacción, por 1 y 0, tal como se observa.
data_r.loc[data_r["Genero"] == "male", "Genero"] = 1
data_r.loc[data_r["Genero"] == "female", "Genero"] = 0
data_r.loc[data_r["Fumador"] == "yes", "Fumador"] = 1
data_r.loc[data_r["Fumador"] == "no", "Fumador"] = 0

In [ ]:
#Ya con la base de datos para clasificación construida, tomamos las variables que nos interesan y las normalizamos, finalmente mostramos la tabla.
df_r=data_r.iloc[:,[0,1,2,3,4,6]]
df_r.insert(0,'bias',np.ones(df_r.shape[0]))
df_r=df_r/df_r.max()
df_r.head()

,bias,Edad,Genero,BMI,Número de hijos,Fumador,Cobro
0,1.0,0.296875,0.0,0.568691,0.0,1.0,0.329819
1,1.0,0.28125,1.0,0.688341,0.2,0.0,0.033706
2,1.0,0.4375,1.0,0.672646,0.6,0.0,0.086913
3,1.0,0.515625,1.0,0.462801,0.0,0.0,0.42943
4,1.0,0.5,1.0,0.588667,0.0,0.0,0.075533


In [ ]:
#Convierto el DataFrame de Pandas a Numpy
df_r=df_r.to_numpy()

In [ ]:
class Neurona():
  #Clase para describir una neurona que puede desarrollar por igual, las actividades de regresión y clasificicación.
  #Los atributos que se definirán serán generales para ambos tipos de problemas que resuelve la neurona.
  #En primer lugar el vector de los deseados, el de las entradas, la cantidad de cada uno, los pesos (aleatorios) y el Error Cuadrático Medio.
  #Las entradas para construir la clase serán únicamente el csv de los datos (puede ser cualquiera, no necesariamente este, siempre y cuando solo tengan una fila de deseados) y el número de iteraciones deseadas.

  def __init__(self,datos,iteraciones):
    self.__deseados=datos[:,None,-1]
    self.__entradas=datos[:,0:datos.shape[1]-1]
    self.__nd=self.__entradas.shape[0]
    self.__ne=self.__entradas.shape[1]
    self.__w=np.random.rand(self.__ne,1)
    self.__y = np.zeros((self.__nd,1))
    self.__nit = iteraciones
    self.__ecm = np.zeros((self.__nit,1))

  @property
  def deseados(self):
    return self.__deseados

  #El algoritmo de clasificación, como entrada, toma los deseados, las entradas de la clase, y además, el alfa.
  def clasificacion(self,alfa):
    for i in range(self.__nit):
      for k in range(self.__nd):
        a = self.__entradas[k,None,:] @ self.__w
        self.__y[k,:]=1/(1+math.exp(-a))
        error = self.__y[k,None,:] - self.__deseados[k,None,:]
        self.__w = self.__w-alfa*error*self.__y[k,None,:]*(1-self.__y[k,None,:])*self.__entradas[k,None,:].T
        print(f"Los pesos w de la iteración {i+1} en la fila {k+1} son: \n {self.__w}")
        self.__ecm[i,:] = self.__ecm[i,None,:] + 1/(2*self.__nd)*error**2


  def regresion(self,alfa):
    for i in range(self.__nit):
      for k in range(self.__nd):
        a = self.__entradas[k,None,:] @ self.__w
        self.__y[k,:]=a
        error = self.__y[k,None,:] - self.__deseados[k,None,:]
        self.__w = self.__w-alfa*error*1*self.__entradas[k,None,:].T
        print(f"Los w de la repetición {i+1} en el lado {k+1} son: {self.__w}")
        self.__ecm[i,:] = self.__ecm[i,None,:] + 1/(2*self.__nd)*error**2

  #Método para obtener la gráfica de la evolución del ECM.
  def graficar(self):
    fig = go.Figure(
    data = [
            go.Scatter(y=(self.__ecm[:,0]))
            ], layout=go.Layout(
        title=go.layout.Title(text="Progresión del Error Cuadrático Medio en función del número de la iteración"))
    )
    fig.update_xaxes(title_text="Número Iteraciones")
    fig.update_yaxes(title_text="ECM")
    fig.show()

  def graficar_yd(self):
    #deseados=[]
    #for i in self.__deseados:
      #deseados.append(i[0])
    #d=np.array(deseados).reshape(-1,1)
    d=self.__deseados[:,0].astype(float)
    fig =px.line(y=[self.__y[:,0],d])
    fig.show()
    
    #plt.plot(self.__y[:,0])
    #plt.plot(self.__deseados[:,0])
    #plt.title("Salida VS Deseados",fontsize=25)
    #plt.xlabel("Número de iteraciones",fontsize=18)
    #plt.ylabel("Valor objetivo",fontsize=18)
    #plt.xlim(0,self.__nd)
    #plt.show()

    
  #Método para obtener la matriz de confusión magnitudinal y porcentual del problema de clasificación. 
  def matriz_confusion(self):
    count_0_0=0
    count_1_0=0
    count_0_1=0
    count_1_1=0
    y=self.__y.copy()
    y_redon=np.round(y)
    for i in range(self.__deseados.T.shape[1]):
      
      if y_redon.T[0,i] == 0 and  self.__deseados.T[0,i]==0:
        count_0_0 +=1
      elif y_redon.T[0,i] == 1 and  self.__deseados.T[0,i]==0:
        count_1_0+=1
      elif y_redon.T[0,i] == 0 and  self.__deseados.T[0,i]==1:
        count_0_1 +=1
      elif y_redon.T[0,i] == 1 and  self.__deseados.T[0,i]==1:
        count_1_1 +=1
    matrizcantidad=pd.DataFrame({'Deseados 0':[count_0_0,count_1_0],'Deseados 1':[count_0_1,count_1_1]},index=['Resultado 0','Resultado 1'])
    print("***Matriz de Confusión por Cantidades***")
    print(matrizcantidad)
    print(" ")
    matrizporcentaje=pd.DataFrame({'Deseados 0':[(count_0_0/(count_0_0+count_1_0+count_0_1+count_1_1))*100,(count_1_0/(count_0_0+count_1_0+count_0_1+count_1_1))*100],'Deseados 1':[(count_0_1/(count_0_0+count_1_0+count_0_1+count_1_1))*100,(count_1_1/(count_0_0+count_1_0+count_0_1+count_1_1))*100]},index=['Resultado 0','Resultado 1'])
    print("***Matriz de Confusión por Porcentajes***")
    print(matrizporcentaje)

In [ ]:
n1=Neurona(df,50)

## Clasificación


In [ ]:
n1.clasificacion(0.5)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [1.6104030601153245]
 [1.005575185127955]]
Los pesos w de la iteración 11 en la fila 15699 son: 
 [[-12.224784476848614]
 [0.41006547919598735]
 [0.4319678383065696]
 [3.4026685464467454]
 [-1.7490297616773607]
 [1.0083129301029776]
 [-0.8378520332682255]
 [8.968289671704442]
 [0.3982232872485328]
 [2.605909809948684]
 [1.9015999473035634]
 [1.61040303869155]
 [1.0055751637041805]]
Los pesos w de la iteración 11 en la fila 15700 son: 
 [[-12.224167009607019]
 [0.41006547919598735]
 [0.43230199704907984]
 [3.402977342024922]
 [-1.7487827747807227]
 [1.0085599169996156]
 [-0.8376050463715875]
 [8.968783645497718]
 [0.3987172610418088]
 [2.606527277190279]
 [1.9022174145451585]
 [1.611020505933145]
 [1.0059456440491377]]
Los pesos w de la iteración 11 en la fila 15701 son: 
 [[-12.248586244100004]
 [0.41006547919598735]
 [0.41707588612992424]
 [3.402423584549966]
 [-1.758550468577917]
 [0.9939083763038243]
 [-0.8473727401687816

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Se han truncado las últimas 5000 líneas del flujo de salida.
 [1.2339843833186175]
 [0.9183962953906101]]
Los pesos w de la iteración 50 en la fila 25620 son: 
 [[-12.11994880182758]
 [0.40134727998761055]
 [0.1993982174373304]
 [3.577958331722255]
 [-2.103394058664567]
 [1.6732052670986195]
 [-0.7778938306561008]
 [9.247301121854774]
 [0.6556179022138078]
 [2.129177046810435]
 [1.6979652937802885]
 [1.2487282890648295]
 [0.9773719183754584]]
Los pesos w de la iteración 50 en la fila 25621 son: 
 [[-12.117700334162981]
 [0.4035957476522082]
 [0.20072084547532903]
 [3.578503414792461]
 [-2.1024946715987283]
 [1.6741046541644586]
 [-0.7756453629915032]
 [9.249549589519372]
 [0.6578663698784054]
 [2.1314255144750325]
 [1.699314374379047]
 [1.249177982597749]
 [0.9796203860400561]]
Los pesos w de la iteración 50 en la fila 25622 son: 
 [[-12.139821533937843]
 [0.4035957476522082]
 [0.19213261497450046]
 [3.5769540873348196]
 [-2.115767391463645]
 [1.6608319342995417]
 [-0.7977665627663647]

In [ ]:
n1.graficar()

In [ ]:
n1.matriz_confusion()

***Matriz de Confusión por Cantidades***
             Deseados 0  Deseados 1
Resultado 0       12553        2927
Resultado 1        2020        8476
 
***Matriz de Confusión por Porcentajes***
             Deseados 0  Deseados 1
Resultado 0   48.325377   11.268094
Resultado 1    7.776409   32.630120


# Regresión

In [ ]:
n2=Neurona(df_r,100)

In [ ]:
n2.regresion(0.5)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [0.0010176180211600126]
 [0.3632165613960164]]
Los w de la repetición 97 en el lado 4 son: [[0.03501289853207301]
 [0.4915768926690859]
 [0.11351616066463985]
 [0.26607008528325216]
 [0.0010176180211600126]
 [0.3632165613960164]]
Los w de la repetición 97 en el lado 5 son: [[-0.20269291437783687]
 [0.37272398621413094]
 [-0.12418965224527004]
 [0.1261405321475368]
 [0.0010176180211600126]
 [0.3632165613960164]]
Los w de la repetición 97 en el lado 6 son: [[-0.18801656861119045]
 [0.3798328411948503]
 [-0.12418965224527004]
 [0.13384067768429747]
 [0.0010176180211600126]
 [0.3632165613960164]]
Los w de la repetición 97 en el lado 7 son: [[-0.19574327616923604]
 [0.374279270137505]
 [-0.12418965224527004]
 [0.12857404293621258]
 [-0.0005277234904491038]
 [0.3632165613960164]]
Los w de la repetición 97 en el lado 8 son: [[-0.17113722821826294]
 [0.3885046416091613]
 [-0.12418965224527004]
 [0.14248704273564172]
 [0.014235905280

In [ ]:
n2.graficar()

In [ ]:
n2.graficar_yd()